### *cis*- and *trans*-QTL mapping with tensorQTL

This notebook provides examples for running *cis*- and *trans*-QTL mapping with tensorQTL, using open-access data from the [GEUVADIS](https://www.ebi.ac.uk/arrayexpress/experiments/E-GEUV-1/) project.

#### Requirements
An environment configured with a GPU and ~50GB of memory.

#### Test dataset

*Note: these files are provided for testing/benchmarking purposes only. They do not constitute an official release from the GEUVADIS project, and no quality-control was applied.*

Genotypes in PLINK2 format (chr18 only), and normalized expression data are available [in this repository](./data/); the full dataset is available at [gs://gtex-resources/test_data/geuvadis](https://console.cloud.google.com/storage/browser/gtex-resources/test_data/geuvadis) ([requester pays](https://cloud.google.com/storage/docs/requester-pays)).

In [1]:
import pandas as pd
import torch
import tensorqtl
from tensorqtl import pgen, cis, trans, post
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"torch: {torch.__version__} (CUDA {torch.version.cuda}), device: {device}")
print(f"pandas {pd.__version__}")

# define paths to data
plink_prefix_path = 'data/GEUVADIS.445_samples.GRCh38.20170504.maf01.filtered.nodup.chr18'
expression_bed = 'data/GEUVADIS.445_samples.expression.bed.gz'
covariates_file = 'data/GEUVADIS.445_samples.covariates.txt'
prefix = 'GEUVADIS.445_samples'

# load phenotypes and covariates
phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expression_bed)
covariates_df = pd.read_csv(covariates_file, sep='\t', index_col=0).T

# PLINK reader for genotypes
pgr = pgen.PgenReader(plink_prefix_path)
genotype_df = pgr.load_genotypes()
variant_df = pgr.variant_df

torch: 2.4.0+cu121 (CUDA 12.1), device: cuda
pandas 2.2.2


### *cis*-QTL: nominal p-values for all variant-phenotype pairs

In [2]:
# map all cis-associations (results for each chromosome are written to file)

# all genes
# cis.map_nominal(genotype_df, variant_df, phenotype_df, phenotype_pos_df, prefix, covariates_df=covariates_df)

# genes on chr18
cis.map_nominal(genotype_df, variant_df,
                phenotype_df.loc[phenotype_pos_df['chr'] == 'chr18'],
                phenotype_pos_df.loc[phenotype_pos_df['chr'] == 'chr18'],
                prefix, covariates_df=covariates_df)

cis-QTL mapping: nominal associations for all variant-phenotype pairs
  * 445 samples
  * 301 phenotypes
  * 26 covariates
  * 367759 variants
  * cis-window: ±1,000,000
  * checking phenotypes: 301/301
  * Computing associations
    Mapping chromosome chr18
    processing phenotype 301/301
    time elapsed: 0.04 min
    * writing output
done.


In [3]:
# load results
pairs_df = pd.read_parquet(f'{prefix}.cis_qtl_pairs.chr18.parquet')
pairs_df.head()

,phenotype_id,variant_id,start_distance,af,ma_samples,ma_count,pval_nominal,slope,slope_se
0,ENSG00000263006.6,chr18_10644_C_G_b38,-98421,0.016854,15,15,0.580873,-0.117761,0.213125
1,ENSG00000263006.6,chr18_10847_C_A_b38,-98218,0.019101,17,17,0.142884,-0.298726,0.203505
2,ENSG00000263006.6,chr18_11275_G_A_b38,-97790,0.024719,22,22,0.745231,0.054619,0.167981
3,ENSG00000263006.6,chr18_11358_G_A_b38,-97707,0.024719,22,22,0.745231,0.054619,0.167981
4,ENSG00000263006.6,chr18_11445_G_A_b38,-97620,0.023596,21,21,0.603276,0.089378,0.171851


### *cis*-QTL: empirical p-values for phenotypes

In [4]:
# all genes
# cis_df = cis.map_cis(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df=covariates_df)

# genes on chr18
cis_df = cis.map_cis(genotype_df, variant_df, 
                     phenotype_df.loc[phenotype_pos_df['chr'] == 'chr18'],
                     phenotype_pos_df.loc[phenotype_pos_df['chr'] == 'chr18'],
                     covariates_df=covariates_df, seed=123456)
# compute q-values (in practice, this must be run on all genes, not a subset)
post.calculate_qvalues(cis_df, fdr=0.05, qvalue_lambda=0.85)

cis-QTL mapping: empirical p-values for phenotypes
  * 445 samples
  * 301 phenotypes
  * 26 covariates
  * 367759 variants
  * cis-window: ±1,000,000
  * using seed 123456
  * checking phenotypes: 301/301
  * computing permutations
    processing phenotype 301/301
  Time elapsed: 0.31 min
done.
Computing q-values
  * Number of phenotypes tested: 301
  * Correlation between Beta-approximated and empirical p-values: 1.0000
  * Calculating q-values with lambda = 0.850
  * Proportion of significant phenotypes (1-pi0): 0.76
  * QTL phenotypes @ FDR 0.05: 205
  * min p-value threshold @ FDR 0.05: 0.135284


In [5]:
cis_df.head()

,num_var,beta_shape1,beta_shape2,true_df,pval_true_df,variant_id,start_distance,end_distance,ma_samples,ma_count,af,pval_nominal,slope,slope_se,pval_perm,pval_beta,qval,pval_nominal_threshold
phenotype_id,,,,,,,,,,,,,,,,,,
ENSG00000263006.6,6120,1.038811,1138.434082,374.660400,8.220950e-40,chr18_112535_G_A_b38,3470,3470,212,251,0.282022,4.050344e-44,0.726425,0.046171,0.000100,3.677735e-38,2.697006e-37,0.000141
ENSG00000101557.14,6355,1.032237,1076.303223,370.176422,5.632806e-11,chr18_210698_T_C_b38,52315,52315,192,222,0.249438,3.505411e-12,-0.191712,0.026749,0.000100,3.498951e-08,3.563747e-08,0.000146
ENSG00000079134.11,6921,1.047219,1155.660156,370.356049,3.888738e-08,chr18_243547_T_A_b38,-24503,-24503,293,383,0.430337,5.473709e-09,-0.122720,0.020602,0.000100,2.743975e-05,1.916427e-05,0.000141
ENSG00000263884.1,6921,1.039806,1152.501587,369.873505,7.681884e-04,chr18_584440_G_C_b38,316292,316292,81,88,0.098876,3.540399e-04,-0.330811,0.091845,0.574843,5.695498e-01,1.577698e-01,0.000139
ENSG00000158270.11,8134,1.054919,1277.927246,369.469086,2.516529e-09,chr18_519222_C_T_b38,18500,18500,108,115,0.129213,2.409717e-10,-0.388277,0.059808,0.000100,1.567348e-06,1.321136e-06,0.000130


### *trans*-QTL mapping

In [6]:
# run mapping
# to limit output size, only associations with p-value <= 1e-5 are returned
trans_df = trans.map_trans(genotype_df, phenotype_df, covariates_df, batch_size=10000,
                           return_sparse=True, pval_threshold=1e-5, maf_threshold=0.05)

trans-QTL mapping
  * 445 samples
  * 19836 phenotypes
  * 26 covariates
  * 367759 variants
    processing batch 37/37
    elapsed time: 0.02 min
  * 210838 variants passed MAF >= 0.05 filtering
done.


In [7]:
# remove cis-associations
trans_df = trans.filter_cis(trans_df, phenotype_pos_df, variant_df, window=5000000)

In [8]:
trans_df.head()

,variant_id,phenotype_id,pval,b,b_se,af
1,chr18_20683_A_G_b38,ENSG00000163900.10,5.012229e-06,0.209540,0.045309,0.179775
3,chr18_27346_G_T_b38,ENSG00000164088.17,7.309937e-06,-0.265623,0.058483,0.123596
11,chr18_43564_G_A_b38,ENSG00000198162.12,1.314060e-07,-0.202922,0.037792,0.093258
12,chr18_43564_G_A_b38,ENSG00000261098.1,8.494569e-06,-0.421968,0.093594,0.093258
13,chr18_43611_C_T_b38,ENSG00000265972.5,1.448981e-06,-0.272301,0.055697,0.135955
